In [13]:
import sys
sys.path.append('/home/jovyan/')
import numba
import copy
# from tests.utils.generate_genome import generate_genome_with_hidden_units
import itertools

In [15]:
from experiments.reporting.report_repository import ReportRepository
from experiments.slack_client import SlackNotifier
from neat.neat_logger import get_neat_logger
from neat.population_engine import EvolutionEngine
from neat.reporting.reports_pyneat import EvolutionReport
from neat.utils import timeit
import math
import random
from itertools import count
import numpy as np

from experiments.logger import logger
from experiments.slack_client import Notifier
from experiments.object_repository.object_repository import ObjectRepository
from experiments.reporting.report_repository import ReportPathFactory
from neat.configuration import get_configuration
from neat.evaluation.evaluation_engine import EvaluationStochasticEngine
from neat.evolution_operators.crossover import Crossover
from neat.evolution_operators.mutation import Mutation
from neat.genome import Genome
from neat.reporting.reports_pyneat import EvolutionReport
from neat.species import SpeciationEngine
from neat.stagnation import Stagnation
from neat.population_engine import PopulationEngine
from neat.utils import timeit
import neat
import tests
from config_files.configuration_utils import create_configuration
from neat.configuration import get_configuration, set_configuration
from neat.evaluation.evaluation_engine import get_dataset
from neat.evaluation.evaluate_simple import evaluate_genome, calculate_prediction_distribution
from neat.genome import Genome
from neat.evolution_operators.backprop_mutation import BackPropMutation
import os
from neat.loss.vi_loss import get_loss
import torch
import pandas as pd

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
from neat.analysis.experiment_data import ExperimentData
from neat.analysis.uncertainty.predictive_distribution import PredictionDistributionEstimator
from neat.analysis.plotting.plot_losses import plot_loss_distribution
from itertools import product
import itertools
from neat.gene import NodeGene, ConnectionGene
from neat.genome import Genome
from config_files.configuration_utils import create_configuration

In [18]:
LOGS_PATH = f'{os.getcwd()}/'
logger = get_neat_logger(path=LOGS_PATH)

In [19]:
PROJECT = 'neuro-evolution'
ALGORITHM_VERSION = 'bayes-neat'
DATASET = 'mnist_downsampled'

# correlation_ids = [f'neat_ft_1_{DATASET}', f'bayesian_neat_ft_1_{DATASET}']
correlation_ids = ['tests']

In [20]:
# from IPython.core.debugger import set_trace
# set_trace()
correlation_id = 'tests'
execution_id = '0cae3bc0-0fb1-4278-b0c5-de286ef8c3ec'
report_repository = ReportRepository.create(project=PROJECT, logs_path=LOGS_PATH)
report = report_repository.get_report(algorithm_version=ALGORITHM_VERSION,
                                                      dataset=DATASET,
                                                      correlation_id=correlation_id,
                                                      execution_id=execution_id)

In [21]:
# report.data['generation_metrics'].keys()

In [22]:
genome_dict = report.data['best_individual']
best_individual_fitness = report.data['best_individual_fitness']
print(f'Fitness of best individual: {best_individual_fitness}')

genome = Genome.from_dict(genome_dict)
config = genome.genome_config
set_configuration(config)

Fitness of best individual: -143.32212829589844


In [24]:
def generate_genome_with_hidden_units(n_input, n_output, n_hidden=3):
    # nodes
    node_genes = {}
    for i in range(n_output + n_hidden):
        node_genes = add_node(node_genes, key=i)

    # connections
    # input to hidden
    connection_genes = {}
    input_hidden_tuples = list(product(list(range(-1, -n_input-1, -1)),
                                       list(range(n_output, n_output+n_hidden))))
    for tuple_ in input_hidden_tuples:
        connection_genes = add_connection(connection_genes, key=tuple_)

    # hidden to output
    hidden_output_tuples = list(product(list(range(n_output, n_output + n_hidden)),
                                        list(range(0, n_output))))
    for tuple_ in hidden_output_tuples:
        connection_genes = add_connection(connection_genes, key=tuple_)

    # initialize genome
    genome = Genome(key=1)
    genome.node_genes = node_genes
    genome.connection_genes = connection_genes
    return genome



def add_node(node_genes, key):
    node_i = NodeGene(key=key)
    node_i.random_initialization()
    node_genes[key] = node_i
    return node_genes


def add_connection(connection_genes, key):
    connection_i = ConnectionGene(key=key)
    connection_i.random_initialization()
    connection_genes[key] = connection_i
    return connection_genes

In [25]:
def test_implementation():
    connections = [(1, 2), (2, 3), (3, 1)]
    if not exist_cycle_numba(connections):
        print(f'Error 1')
    

#     def test_exists_cycle_positive_b(self):
    connections = [(1, 2), (2, 3), (1, 4), (4, 3), (3, 1)]
    if not exist_cycle_numba(connections):
        print(f'Error 2')
#         self.assertEqual(True, exist_cycle_numba(connections))

#     def test_exists_cycle_when_negative_a(self):
    connections = [(1, 2), (2, 3), (3, 4)]
    if exist_cycle_numba(connections):
        print(f'Error 3')
#         self.assertEqual(False, exist_cycle_numba(connections))

#     def test_exists_cycle_when_negative_b(self):
    connections = [(1, 2), (2, 3), (1, 4), (4, 3), (3, 5)]
    if exist_cycle_numba(connections):
        print(f'Error 4')
#         self.assertEqual(False, exist_cycle_numba(connections))

#     def test_exists_cycle_when_negative_c(self):
    connections = [(-1, 1), (-1, 2), (-1, 3), (-2, 1), (-2, 2), (-2, 3), (1, 0), (2, 0), (3, 0), (1, 2)]
    if exist_cycle_numba(connections):
        print(f'Error 5')
#         self.assertEqual(False, exist_cycle_numba(connections))

#     def test_self_recursive(self):
    connections = [(1, 1)]
    if not exist_cycle_numba(connections):
        print(f'Error 6')
#         self.assertEqual(True, exist_cycle_numba(connections))

#     def test_self_recursive_b(self):
    connections = [(-1, 1), (-1, 2), (-1, 3), (-2, 1),  (-2, 2), (-2, 3), (1, 0), (2, 0), (3, 0), (2, 2)]
    if not exist_cycle_numba(connections):
        print(f'Error 7')
#         self.assertEqual(True, exist_cycle_numba(connections))

In [28]:
# @numba.jit(nopython=True)
@numba.jit
def _go_through_graph(node_in, graph, past=[]):
    if node_in in graph.keys():
        for node in graph[node_in]:
            if node in past:
                return True
            else:
                past_copy = copy.deepcopy(past)
                past_copy.append(node_in)
                if _go_through_graph(node_in=node, graph=graph, past=past_copy):
                    return True
    else:
        return False
        

def exist_cycle_numba(connections: list) -> bool:
    # change data structure
    con = _get_connections_per_node(connections)
#     print(con)
    for node_in, nodes_out in con.items():
        if _go_through_graph(node_in, graph=con, past=[]):
            return True
    return False


def _get_connections_per_node(connections: list, inverse_order=False):
    '''
    :param connections: eg. ((-1, 1), (1, 2), (2, 3), (2, 4))
    :param inverse_order: whether it follows the input to output direction or the output to input direction
    :return: {-1: [1], 1: [2], 2: [3, 4]
    '''
    con = {}
    for connection in connections:
        input_node_key, output_node_key = connection
        if inverse_order:
            output_node_key, input_node_key = connection
        if input_node_key in con:
            con[input_node_key].append(output_node_key)
        else:
            con[input_node_key] = [output_node_key]
    return con

In [29]:
%timeit test_implementation()

<ipython-input-28-6985f868a596>:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "_go_through_graph" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-28-6985f868a596> (4)

File "<ipython-input-28-6985f868a596>", line 4:
def _go_through_graph(node_in, graph, past=[]):
    if node_in in graph.keys():
    ^

  @numba.jit
/opt/conda/lib/python3.7/site-packages/numba/compiler.py:742: NumbaWarning: Function "_go_through_graph" was compiled in object mode without forceobj=True.

File "<ipython-input-28-6985f868a596>", line 3:
@numba.jit
def _go_through_graph(node_in, graph, past=[]):
^

  self.func_ir.loc))
/opt/conda/lib/python3.7/site-packages/numba/compiler.py:751: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/la

1.57 ms ± 91 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


/opt/conda/lib/python3.7/site-packages/numba/compiler.py:742: NumbaWarning: Function "_go_through_graph" was compiled in object mode without forceobj=True.

File "<ipython-input-28-6985f868a596>", line 3:
@numba.jit
def _go_through_graph(node_in, graph, past=[]):
^

  self.func_ir.loc))
/opt/conda/lib/python3.7/site-packages/numba/compiler.py:751: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "<ipython-input-28-6985f868a596>", line 3:
@numba.jit
def _go_through_graph(node_in, graph, past=[]):
^

  warnings.warn(errors.NumbaDeprecationWarning(msg, self.func_ir.loc))


In [16]:
def _remove_connection_that_introduces_cycles(genome: Genome, possible_connection_set: set) -> set:
    connections_to_remove = []
    for connection in possible_connection_set:
        connections = list(genome.connection_genes.keys()) + [connection]

        if exist_cycle_numba(connections=connections):
            connections_to_remove.append(connection)
#     logger.debug(f'connections that introduce cycles: {connections_to_remove}')
    possible_connection_set -= set(connections_to_remove)
    return possible_connection_set

In [12]:
# config = create_configuration(filename='/regression-miso.json')
# genome = generate_genome_with_hidden_units(n_input=5, n_output=10, n_hidden=100)
out_node_key = 5

# all nodes
possible_input_keys_set = set(genome.node_genes.keys()).union(set(genome.get_input_nodes_keys()))

# no connection between two output nodes
possible_input_keys_set -= set(genome.get_output_nodes_keys())

if config.feed_forward:
    # avoid self-recurrency
    possible_input_keys_set -= {out_node_key}
    # pass

# REMOVE POSSIBLE CONNECTIONS
possible_connection_set = set(itertools.product(list(possible_input_keys_set), [out_node_key]))

# remove already existing connections: don't duplicate connections
possible_connection_set -= set(genome.connection_genes.keys())

NameError: name 'itertools' is not defined

In [19]:
%timeit _remove_connection_that_introduces_cycles(genome, possible_connection_set)

172 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
